## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

## Auditing

> - Some Stuff (Query 1)
>     - More Stuff (Query 2)
>     - Some Info
>         - More Info- (Query 3.1)
>         - Even More Info IP - (Query 3.2)

In [ ]:
/* Database audit specifications in a SQL Server audit on a server instance for all databases */
SELECT TOP 1000 
	'[database_audit_specification_details]' AS TableName
	,[database_specification_id]
	,[audit_action_id]
	,[audit_action_name]
	,[class]
	,[class_desc]
	,[major_id]
	,[minor_id]
	,[audited_principal_id]
	,[audited_result]
	,[is_group]
FROM [master].[sys].[database_audit_specification_details]


In [ ]:
/* Contains information about the database audit specifications in a SQL Server audit on a server instance. */
SELECT TOP 1000 
	'[database_audit_specifications]' AS TableName
	,[database_specification_id]
	,[name]
	,[create_date]
	,[modify_date]
	,[audit_guid]
	,[is_state_enabled]
FROM [master].[sys].[database_audit_specifications]


In [ ]:
/* Returns a row for every audit action that can be reported in the audit log and every audit action group  */
--that can be configured as part of SQL Server Audit.  
SELECT TOP 1000 
	'[dm_audit_actions]' AS TableName
	,[action_id]
	,[name]
	,[class_desc]
	,[covering_action_name]
	,[parent_class_desc]
	,[covering_parent_action_name]
	,[configuration_level]
	,[containing_group_name]
	,[action_in_log]
FROM [master].[sys].[dm_audit_actions]


In [ ]:
/* Returns a table that maps the class_type field in the audit log to the class_desc field in sys.dm_audit_actions  */
SELECT TOP 1000 
	'[dm_audit_class_type_map]' AS TableName
	,[class_type]
	,[class_type_desc]
	,[securable_class_desc]
FROM [master].[sys].[dm_audit_class_type_map]


In [ ]:
/* Returns a row for each server audit indicating the current state of the audit. */
SELECT TOP 1000 
	'[dm_server_audit_status]' AS TableName
	,[audit_id]
	,[name]
	,[status]
	,[status_desc]
	,[status_time]
	,[event_session_address]
	,[audit_file_path]
	,[audit_file_size]
FROM [master].[sys].[dm_server_audit_status]


In [ ]:
/* Contains information about the server audit specification details (actions) in a SQL Server audit on a server instance.*/
SELECT TOP 1000 
	'[server_audit_specification_details]' AS TableName
	,[server_specification_id]
	,[audit_action_id]
	,[audit_action_name]
	,[class]
	,[class_desc]
	,[major_id]
	,[minor_id]
	,[audited_principal_id]
	,[audited_result]
	,[is_group]
FROM [master].[sys].[server_audit_specification_details]


In [ ]:
/* Contains information about the server audit specifications in a SQL Server audit on a server instance */
SELECT TOP 1000 
	'server_audit_specifications' AS TableName
	,[server_specification_id]
	,[name]
	,[create_date]
	,[modify_date]
	,[audit_guid]
	,[is_state_enabled]
FROM [master].[sys].[server_audit_specifications]


In [ ]:

/* Contains one row for each SQL Server audit in a server instance */
SELECT TOP 1000 
	'[server_audits]' AS TableName
	,[audit_id]
	,[name]
	,[audit_guid]
	,[create_date]
	,[modify_date]
	,[principal_id]
	,[type]
	,[type_desc]
	,[on_failure]
	,[on_failure_desc]
	,[is_state_enabled]
	,[queue_delay]
FROM [master].[sys].[server_audits]


In [ ]:
/* Contains extended information about the file audit type in a SQL Server audit on a server instance */
SELECT TOP 1000 
	'[server_file_audits]' AS TableName
	,[audit_id]
	,[name]
	,[audit_guid]
	,[create_date]
	,[modify_date]
	,[principal_id]
	,[type]
	,[type_desc]
	,[on_failure]
	,[on_failure_desc]
	,[is_state_enabled]
	,[queue_delay]
	,[max_file_size]
	,[max_rollover_files]
	,[reserve_disk_space]
	,[log_file_path]
	,[log_file_name]
FROM [master].[sys].[server_file_audits]


In [ ]:
/* Get Audit information for specific Audits */
SELECT AuditFile.*
FROM sys.dm_server_audit_status AS AuditStatus
CROSS APPLY sys.fn_get_audit_file(AuditStatus.audit_file_path, DEFAULT, DEFAULT) AS AuditFile



In [ ]:
/* Disable and Drop Audit */
USE [master]
GO

DECLARE @Audit_Name sysname ='server_file_audits',
	    @sqlcmd VARCHAR(MAX)

IF EXISTS(SELECT name FROM sys.server_audits WHERE is_state_enabled = 1 AND Name =@Audit_Name)
BEGIN
	SET @sqlcmd ='ALTER SERVER AUDIT [' + @Audit_Name + '] WITH (STATE = OFF)'
	EXEC (@sqlcmd)

    SET @sqlcmd=  'DROP SERVER AUDIT ['+ @Audit_Name + ']'
    EXEC (@sqlcmd)
END


In [ ]:
/* Drop audit that exists and is disabled */
USE [master]
GO

DECLARE @Audit_Name sysname ='JobHistory_Audit',
	    @sqlcmd VARCHAR(MAX)

IF EXISTS(SELECT name FROM sys.server_audits WHERE is_state_enabled = 0 AND Name =@Audit_Name)
BEGIN
	SET @sqlcmd=  'DROP SERVER AUDIT ['+ @Audit_Name + ']'
    EXEC (@sqlcmd)
END


In [ ]:

SELECT name FROM sys.server_audits WHERE is_state_enabled = 1 AND Name =$Audit_Name

In [ ]:
/* 02 Audit Job changes.sql  */
--=====================================
--Create Job history Audits
--=====================================

USE [master]
GO


CREATE SERVER AUDIT [JobHistory_Audit] TO FILE (
	FILEPATH = N'C:\Audit'
	,MAXSIZE = 0 MB
	,MAX_ROLLOVER_FILES = 2147483647
	,RESERVE_DISK_SPACE = OFF
	)
	WITH (
			QUEUE_DELAY = 1000
			,ON_FAILURE = CONTINUE
			)
GO

USE [msdb]
GO



CREATE DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-EnterDate]
FOR SERVER AUDIT [JobHistory_Audit] ADD (
		UPDATE ON OBJECT::[dbo].[sysjobs] BY [public]
		)
	,ADD (
		INSERT ON OBJECT::[dbo].[sysjobs] BY [public]
		)
	,ADD (
		DELETE ON OBJECT::[dbo].[sysjobs] BY [public]
		)
GO

ALTER DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-EnterDate]
	WITH (STATE = ON)

USE master
GO

ALTER SERVER AUDIT [JobHistory_Audit]
	WITH (STATE = ON)
GO

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobHistory*', DEFAULT, DEFAULT)
GO

--Sysjobsteps
USE MASTER
GO

CREATE SERVER AUDIT [JobStepsHistory_Audit] TO FILE (
	FILEPATH = N'C:\Audit'
	,MAXSIZE = 0 MB
	,MAX_ROLLOVER_FILES = 2147483647
	,RESERVE_DISK_SPACE = OFF
	)
	WITH (
			QUEUE_DELAY = 1000
			,ON_FAILURE = CONTINUE
			)
GO

USE MSDB
GO

CREATE DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-JobSteps]
FOR SERVER AUDIT [JobStepsHistory_Audit] ADD (
		UPDATE ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)
	,ADD (
		INSERT ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)
	,ADD (
		DELETE ON OBJECT::[dbo].[sysjobsteps] BY [public]
		)

ALTER DATABASE AUDIT SPECIFICATION [DatabaseAuditSpecification-JobSteps]
	WITH (STATE = ON)

USE master
GO

ALTER SERVER AUDIT [JobStepsHistory_Audit]
	WITH (STATE = ON)
GO

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobStepsHistory*', DEFAULT, DEFAULT)

SELECT statement
	,*
FROM fn_get_audit_file('C:\Audit\jobHistory*', DEFAULT, DEFAULT)


## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)